# Data export

This notebook extracts the predictions and user-labeled instances (if configured) from a chosen SLEAP dataset file and exports them in a comma-separated values file that can be used for subsequent analysis. The structure of the output file is as follows,

|Field name|Data type|Description|
|-|-|:-|
|video_path|String|The absolute path to a video containing one or more instances of ants.|
|video_width|Integer|The width of the video in pixels.|
|video_height|Integer|The height of the video in pixels.|
|frame_idx|Integer|The index for a frame in the video given by video_path.|
|instance_idx|Integer|The index for an instance in a frame (not applicable between frames).|
|track_id|String|The name of the track that the instance belongs to, or blank if the instance does not belong to a track.|
|part_name|String|The technical name for a body part in an instance.|
|predicted|Boolean|True if the record is the predicted position of a body part, False if the record is the ground truth for a body part position.|
|x_pos|Float|The X-coordinate of the body part in pixels.|
|y_pos|Float|The Y-coordinate of the body part in pixels.|

Enter values for each variable in the first cell below according to your setup and requirements. Each variable and corresponding value should look like the following,

`variable_name = value`

After entering values for all variables, click Cell > Run All in the top menu bar to execute the notebook and generate the output file.

Note that the 'r' before the opening quotation mark for the `dataset_path` and `output_path` variables is required to ensure that backslashes in the folder path don't cause issues during processing, as the backslash is usually used to denote an "escaped" character in character strings.

## Parameters

In [1]:
# Enter the file path for the SLEAP dataset that 
# e.g. r"D:\Documents\Ant-posture\datasets\main.v001.slp"
dataset_path = r""

# Enter the file path where the output file should be saved to
# e.g. r"D:\Documents\Ant-posture\postures.csv"
output_path = r""

# Set to True (without quotation marks) if predictions should be exported only (i.e. not user-labelled instances).
# Otherwise, set to False if predictions and user-labelled instances should exported together
predictions_only = True

## Code execution

In [2]:
# Import the SLEAP Python module to load data from SLEAP dataset files
import sleap

# Load the labels and predictions from the input dataset
dataset = sleap.load_file(dataset_path)

# Raise an error if there are no labelled frames or predictions to export
if len(dataset.labeled_frames) == 0:
    raise RuntimeError("Dataset has no data to export, run inference on the dataset first in SLEAP")

In [3]:
# Sort the frames by frame index
sorted_frames = sorted(dataset.labeled_frames, key=lambda x: x.frame_idx)

# Write data to the output file
try:
    with open(output_path, 'w') as file:
        file.write('video_path,video_width,video_height,frame_idx,instance_idx,track_id,part_name,predicted,x_pos,y_pos\n')

        for frame in sorted_frames:
            if not predictions_only:
                for instance_idx, instance in enumerate(frame.user_instances):
                    for point_idx, point in enumerate(instance.points_array):
                        file.write(f'{frame.video.filename},{frame.video.width},{frame.video.height},{frame.frame_idx},{instance_idx},{instance.track.name if instance.track else ""},{dataset.skeleton.nodes[point_idx].name},False,{point[0]},{point[1]}\n')

            for instance_idx, instance in enumerate(frame.predicted_instances):
                for point_idx, point in enumerate(instance.points_array):
                    file.write(f'{frame.video.filename},{frame.video.width},{frame.video.height},{frame.frame_idx},{instance_idx},{instance.track.name if instance.track else ""},{dataset.skeleton.nodes[point_idx].name},True,{point[0]},{point[1]}\n')
except FileNotFoundError:
    print(e)
    print("Make sure that missing videos have been replaced if the project file was created on another machine")